In [1]:
import glob
import os
import pandas as pd
import re
import ast
from collections import Counter

In [262]:
dataset = load_dataset("glue", "qnli")
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['question', 'sentence', 'label', 'idx'],
        num_rows: 104743
    })
    validation: Dataset({
        features: ['question', 'sentence', 'label', 'idx'],
        num_rows: 5463
    })
    test: Dataset({
        features: ['question', 'sentence', 'label', 'idx'],
        num_rows: 5463
    })
})


In [265]:
sen='For example, Joseph Haas was arrested for allegedly sending an email to the Lebanon, New Hampshire city councilors stating, "Wise up or die."'
if sen in dataset['validation']['sentence'] or dataset['test']['sentence']:
    print(sen)

For example, Joseph Haas was arrested for allegedly sending an email to the Lebanon, New Hampshire city councilors stating, "Wise up or die."


In [ ]:
import json, ast, re
import pandas as pd
from collections import Counter


OUTPUT_BLOCK = re.compile(r"(?:Sample\s+)?Output\s*:\s*(\[[\s\S]*?\])", re.IGNORECASE)

def normalize_text(s: str) -> str:
    
    s = re.sub(r"\s+", " ", str(s))
    s = re.sub(r"\s+([.,;:])", r"\1", s)
    s = re.sub(r"([.,;:])\s*", r"\1 ", s)
    return s.strip()

def _loads_list_any(s: str):
    """JSON -> ast.literal_eval parsing if fail None."""
    for loader in (json.loads, ast.literal_eval):
        try:
            obj = loader(s)
            if isinstance(obj, list):
                return obj
        except Exception:
            pass
    return None

def _is_empty_like(x) -> bool:
    """NaN/None/'nan'/'null'/'none'/'[]' ->empty."""
    if x is None:
        return True
    try:
        
        from math import isnan
        if isinstance(x, float) and isnan(x):
            return True
    except Exception:
        pass
    s = str(x).strip()
    if s == "": return True
    if s.lower() in {"nan", "null", "none"}: return True
    if s == "[]": return True
    return False
def parse_pred(cell):
    """
    pred:
      1) 'Output:' if block only use the content in [ ... ]
      2) parsing ["..."] 리스트로 파싱 시도(LLM 출력 호환)
      3) 1 fail ,2 fail: empty→[],or single string
    return: (list[str], ok:bool)  # ok=True, parsing success
    """
    if _is_empty_like(cell):
        return []
    s = str(cell).strip()

    m = OUTPUT_BLOCK.search(s)
    if m:
        block = m.group(1).strip()
        obj = _loads_list_any(block)
        if obj is None:
            return [], False
        return [normalize_text(str(it)) for it in obj]

    obj = _loads_list_any(s)
    if obj is not None:
        return [normalize_text(str(it)) for it in obj]

    return [normalize_text(s)]
def parse_gold(cell):
    """
    gold:
      - if list of json -> parsing 
      - or empty -> []], or single string 
    return: (list[str], ok:bool)
    """
    if _is_empty_like(cell):
        return []
    if isinstance(cell, list):
        return [normalize_text(str(x)) for x in cell]
    s = str(cell).strip()
    obj = _loads_list_any(s)
    if obj is not None:
        return [normalize_text(str(it)) for it in obj]
    return [normalize_text(s)]

def processing_from_file(df: pd.DataFrame):
    df = df.copy()
    if "idx" not in df.columns:
        df = df.reset_index().rename(columns={"index": "idx"})
    def parse_row(row):
        gold = parse_gold(row.get("gold", "") )
        pred = parse_pred(row.get("pred", "") )
        tag  = row.get('tag', None)
        return pd.Series({
            "idx": row["idx"],
            "gold": gold,
            "pred": pred,
            "tag": tag
        })
    df_parse = df.apply(parse_row, axis=1)
    return df_parse





In [ ]:
-Llama
1.llama_no_training
C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\llama_no_training
2.llama_sft20
C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\dpo_sample20_sft\strong_prompt_soft_model
3.llama_curriculum_dpo
C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\curriculum_dpo\strong_prompt_soft_model
4.llama_dpo
C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\dpo_only\strong_prompt_strong_model
5.llama_sft20+curriculum+dpo(soft)
C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\dpo_sample20\strong_prompt_soft_model
6.llama sft20+curriculum+dpo(strong)
C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\dpo_sample_compare\strong_prompt_strong_model
-Phi
1.phi_no_training
C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\phi\phi_no_training
2.phi_sft20
C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\phi\phi_sample_sft20\strong_prompt_strong_model
3-1.phi_curriculum_dpo(strong)
C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\phi\phi_curriculum_dpo_no_sft\strong_prompt_strong_model
3-2.phi_curriculum_dpo(soft model)
C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\phi\phi_curriculum_dpo_compare\strong_prompt_soft_model
4. Dpo
C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\phi\phi_dpo\strong_prompt_strong_model
5.phi_sft20+curriculum+dpo
C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\phi\phi_curriculum_dpo_sample20\strong_prompt_strong_model


In [345]:
current_path=r"C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\phi\sft_curriculumnodpo"
pattern=os.path.join(current_path,"shard*.csv")
file_paths = sorted(glob.glob(pattern))
postprocessing_paths=os.path.join(current_path,"post_processing.csv")
#saved_result=r"C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\phi\phi_curriculum_dpo_no_sft\strong_prompt_strong_model"

In [346]:
file_paths

['C:\\Users\\haeun\\Downloads\\hw\\computer_share\\dpo_models\\output\\phi\\sft_curriculumnodpo\\shard_0.csv',
 'C:\\Users\\haeun\\Downloads\\hw\\computer_share\\dpo_models\\output\\phi\\sft_curriculumnodpo\\shard_1.csv',
 'C:\\Users\\haeun\\Downloads\\hw\\computer_share\\dpo_models\\output\\phi\\sft_curriculumnodpo\\shard_2.csv',
 'C:\\Users\\haeun\\Downloads\\hw\\computer_share\\dpo_models\\output\\phi\\sft_curriculumnodpo\\shard_3.csv',
 'C:\\Users\\haeun\\Downloads\\hw\\computer_share\\dpo_models\\output\\phi\\sft_curriculumnodpo\\shard_4.csv',
 'C:\\Users\\haeun\\Downloads\\hw\\computer_share\\dpo_models\\output\\phi\\sft_curriculumnodpo\\shard_5.csv',
 'C:\\Users\\haeun\\Downloads\\hw\\computer_share\\dpo_models\\output\\phi\\sft_curriculumnodpo\\shard_6.csv',
 'C:\\Users\\haeun\\Downloads\\hw\\computer_share\\dpo_models\\output\\phi\\sft_curriculumnodpo\\shard_7.csv',
 'C:\\Users\\haeun\\Downloads\\hw\\computer_share\\dpo_models\\output\\phi\\sft_curriculumnodpo\\shard_8.csv',
 

In [ ]:

#"C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\curriculum_strong_4targets_left"

combined =pd.concat([pd.read_csv(path) for path in file_paths],ignore_index=True)
  

tag_map ={ 0 :'C', 1 : "DC" , 2 : 'FC', 3 : 'T',4 : 'CT'}
combined['tag'] = combined.index%5
combined['tag'] = combined['tag'].map(tag_map)
combined_processing=processing_from_file(combined)

print(combined_processing)



         idx                                               gold  \
0          0  [As of that day, the new constitution heraldin...   
1          1                                                 []   
2          2                                                 []   
3          3  [As of that day, the new constitution heraldin...   
4          4                                                 []   
...      ...                                                ...   
30665  30665  [" Official " reconstructions have also been r...   
30666  30666                                                 []   
30667  30667                                                 []   
30668  30668  [" Official " reconstructions have also been r...   
30669  30669                                                 []   

                                                    pred tag  
0      [As of that day, the new constitution heraldin...   C  
1      [As of that day, the new constitution heraldin...  DC  
2     

In [349]:
combined_processing.to_csv(postprocessing_paths)

In [28]:
file_paths={
"example":r"C:\Users\haeun\Downloads\hw\langchain-kr\Lingustic\model_finetune\two_step\langgraph_project\llama_example.partial.csv",
"vanila":r"C:\Users\haeun\Downloads\hw\langchain-kr\Lingustic\model_finetune\two_step\langgraph_project\llama_vanila.partial.csv",
"parse":r"C:\Users\haeun\Downloads\hw\langchain-kr\Lingustic\model_finetune\two_step\langgraph_project\llama_parse.partial.csv",
"annotated":r"C:\Users\haeun\Downloads\hw\langchain-kr\Lingustic\model_finetune\two_step\langgraph_project\llama_annotated.partial.csv"
}

LLama
1.curriculum_Dpo
C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\curriculum_dpo\strong_prompt_soft_model
2.dpo_only
C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\dpo_only\strong_prompt_strong_model\*.csv
3.sample_20
C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\dpo_sample20\strong_prompt_soft_model
4-1 dpo-sample-compare(strong prompt soft model)
C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\dpo_sample_compare\strong_prompt_soft_model(now)
4-2 dpo_sample_compare(strong prompt strong model)
5. dpo_sample 20 sft
C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\dpo_sample20_sft\strong_prompt_soft_model
phi
6.phi_no_training
C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\phi\phi_no_training
1.sample_20
C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\phi\phi_sample_sft20\strong_prompt_strong_model
2-1. curriculum + dpoC:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\phi\phi_curriculum_dpo_no_sft\strong_prompt_strong_model
2-2 curriculum+dpo
C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\phi\phi_curriculum_dpo_compare\strong_prompt_soft_model
3.sample 20 sft+ curriculum+ dpo
C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\phi\phi_curriculum_dpo_sample20\strong_prompt_strong_model
4. dpo_Only
C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\phi\phi_dpo\strong_prompt_strong_model
5.llama no training

C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\llama_no_training



# token based

In [ ]:
saved_results=[r"C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\llama_no_training",
r"C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\dpo_sample20_sft\strong_prompt_soft_model",
r"C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\curriculum_dpo\strong_prompt_soft_model",
r"C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\dpo_only\strong_prompt_strong_model",
r"C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\dpo_sample20\strong_prompt_soft_model",
r"C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\dpo_sample_compare\strong_prompt_strong_model",
r"C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\phi\phi_no_training",
r"C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\phi\phi_sample_sft20\strong_prompt_strong_model",
r"C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\phi\phi_curriculum_dpo_no_sft\strong_prompt_strong_model",
r"C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\phi\phi_curriculum_dpo_compare\strong_prompt_soft_model",
r"C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\phi\phi_dpo\strong_prompt_strong_model",
r"C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\phi\phi_curriculum_dpo_sample20\strong_prompt_strong_model",
r"C:\Users\haeun\Downloads\hw\computer_share\after_final\eval_result_llama_finetune",
r"C:\Users\haeun\Downloads\hw\computer_share\after_final\eval_result_phi_finetune"
]

In [353]:
saved=r"C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\phi\sft_curriculumnodpo\post_processing.csv"
saved_file="\\".join(saved.split("\\")[:-1])
saved_file

'C:\\Users\\haeun\\Downloads\\hw\\computer_share\\dpo_models\\output\\phi\\sft_curriculumnodpo'

In [ ]:
#llama 
import json, ast, re, argparse
import pandas as pd
from collections import Counter
import math
import os
from datasets import load_dataset
from typing import List, Tuple
OUTPUT_BLOCK = re.compile(r"(?:Sample\s+)?Output\s*:\s*(\[[\s\S]*?\])", re.IGNORECASE)
WORD = re.compile(r"\w+('\w+)?", re.UNICODE)
def normalize(s: str) -> str:
    s = re.sub(r"\s+", " ", str(s))
    s = re.sub(r"\s+([.,;:])", r"\1", s)
    s = re.sub(r"([.,;:])\s*", r"\1 ", s)
    return s.strip()

def is_empty_like(x):
    if x is None: return True
    if isinstance(x, float) and math.isnan(x): return True
    s = str(x).strip()
    return s == "" or s.lower() in {"nan","null","none"}

def loads_list_any(s: str):
    for loader in (json.loads, ast.literal_eval):
        try:
            obj = loader(s)
            if isinstance(obj, list):
                return obj
        except Exception:
            pass
    return None
def to_list(cell, is_pred=False):
   
    if is_empty_like(cell): 
        return []
    if isinstance(cell, list):
        return [normalize(str(v)) for v in cell]

    s = str(cell).strip()

    
    if is_pred:
        m = OUTPUT_BLOCK.search(s)
        if m:
            s = m.group(1).strip()  

    # (JSON/리터럴) parsing
    obj = loads_list_any(s)
    if isinstance(obj, list):
        return [normalize(str(v)) for v in obj]

    
    return [normalize(s)]


def tokenize_spans(spans, lower=True):
 
    toks = []
    for s in spans:
        s = s if isinstance(s, str) else str(s)
        if lower:
            s = s.lower()
        toks.extend(m.group(0) for m in WORD.finditer(s))
    return toks

def prf_from_counts(tp, fp, fn):
    prec = tp / (tp + fp) if tp + fp > 0 else 0.0
    rec  = tp / (tp + fn) if tp + fn > 0 else 0.0
    f1   = 2 * prec * rec / (prec + rec) if prec + rec > 0 else 0.0
    return prec, rec, f1
def row_metrics_span(gold_spans, pred_spans,iou_tau=None, ):
    
    if not gold_spans and not pred_spans:
        return 1.0, 1.0, 1.0
    gc, pc = Counter(gold_spans), Counter(pred_spans)
    tp = sum((gc & pc).values())
    fp = sum(pc.values()) - tp
    fn = sum(gc.values()) - tp
    return prf_from_counts(tp, fp, fn)
def row_metrics_token(gold_spans, pred_spans,):
    if not gold_spans and not pred_spans:
        return 1.0, 1.0, 1.0,1.0
    gt, pt = Counter(tokenize_spans(gold_spans)), Counter(tokenize_spans(pred_spans))
    tp = sum((gt & pt).values())
    fp = sum(pt.values()) - tp
    fn = sum(gt.values()) - tp
    precision, recall, f1 = prf_from_counts(tp, fp, fn)
    jaccard = tp / (tp + fp + fn) if (tp + fp + fn) else 1.0
    return precision, recall, f1, jaccard
def load_test_with_sentence():
    ds = load_dataset("haeunkim/curriculum_learning", split="test")
    df_test = ds.to_pandas()
    # sentence column extract: priority in order: 'input' → 'sentence' → 'text' 등
    for col in ["input"]:
        if col in df_test.columns:
            sent_col = col
            break
    else:
        raise ValueError(f"cannot find columns. columns={list(df_test.columns)[:10]}")
    out = df_test[[sent_col]].copy()
    out = out.rename(columns={sent_col: "sentence"})
    out["idx"] = range(len(out))
    return out
def all_occurs(text: str, span: str, *, overlapping=True, ignore_case=False) -> List[Tuple[int,int]]:
    
    if not span:
        return []
    flags = re.DOTALL | (re.IGNORECASE if ignore_case else 0)
    lit = re.escape(span)  
    if overlapping:
        rx = re.compile(rf"(?={lit})", flags)
        return [(m.start(), m.start()+len(span)) for m in rx.finditer(text)]
    else:
        rx = re.compile(lit, flags)
        return [(m.start(), m.end()) for m in rx.finditer(text)]
def tagwise_both(df, gold_col="gold" , pred_col="pred", tag_col="tag"):
    """
    return:
      rows_df: rowsise p/r/f1 (span & token)
      tag_span: macro(average) PRF + n
      tag_token: macro(average) PRF + n
    """
    test_df = load_test_with_sentence()
    df = df.merge(test_df, on="idx", how="left")
    rows = []
    for _, r in df.iterrows():
        gold = to_list(r.get(gold_col, ""), is_pred=False)
        pred = to_list(r.get(pred_col, ""), is_pred=True)
        p_s, r_s, f1_s = row_metrics_span(gold, pred)
        p_t, r_t, f1_t ,J= row_metrics_token(gold, pred)
        rows.append({
            "tag": r.get(tag_col, None),
            "precision_span": p_s, "recall_span": r_s, "f1_span": f1_s,
            "precision_tok":  p_t, "recall_tok":  r_t, "f1_tok":  f1_t,
            "jaccard_tok": J
        })
    rows_df = pd.DataFrame(rows)

    # macro averate per tag
    tag_span = rows_df.groupby("tag")[["precision_span","recall_span","f1_span"]].mean() \
                      .rename(columns={"precision_span":"precision","recall_span":"recall","f1_span":"f1"})
    tag_token = rows_df.groupby("tag")[["precision_tok","recall_tok","f1_tok","jaccard_tok"]].mean() \
                       .rename(columns={"precision_tok":"precision","recall_tok":"recall","f1_tok":"f1","jaccard_tok":"jaccard"})
    
    counts = rows_df.groupby("tag").size().rename("count")
    tag_span = tag_span.join(counts)
    tag_token = tag_token.join(counts)
    
    return rows_df, tag_span, tag_token




df=pd.read_csv(saved)
rows_df, tag_span, tag_token = tagwise_both(df)
print(saved)
print("[TAG span macro]\n", tag_span)
print("[TAG token macro]\n", tag_token)
rows_df.to_csv(os.path.join(saved_file,"rows_per_example.csv"), index=False)
tag_span.to_csv(os.path.join(saved_file,"tag_span_macro.csv"))
tag_token.to_csv(os.path.join(saved_file,"tag_token_macro.csv"))
   



C:\Users\haeun\Downloads\hw\computer_share\dpo_models\output\phi\sft_curriculumnodpo\post_processing.csv
[TAG span macro]
      precision    recall        f1  count
tag                                      
C     0.345578  0.267790  0.291484   6134
CT    0.843128  0.803195  0.815464   6134
DC    0.829162  0.824474  0.824194   6134
FC    0.989077  0.989077  0.989077   6134
T     0.810483  0.810564  0.810510   6134
[TAG token macro]
      precision    recall        f1   jaccard  count
tag                                                
C     0.809762  0.664270  0.689352  0.599629   6134
CT    0.910472  0.854701  0.871824  0.852960   6134
DC    0.883831  0.885906  0.877438  0.862949   6134
FC    0.989077  0.989077  0.989077  0.989240   6134
T     0.891902  0.888252  0.889979  0.884593   6134


In [241]:
import random
#negative sample

def left_trim(span: str):
    toks = span.split()
    return " ".join(toks[1:]) if len(toks) >= 2 else ""

def right_trim(span: str):
    toks = span.split()
    return " ".join(toks[:-1]) if len(toks) >= 2 else ""

def both_trim(span: str):
    toks = span.split()
    return " ".join(toks[1:-1]) if len(toks) >= 3 else ""
def get_negative_sample(sentence,chosen):
   
    choice=random.randint(0,4)
    len_sentence=len(sentence)
    if choice == 0:
        
        repeat =random.randint(0,5)
        rejected=[sentence] * repeat
        
        out =  json.dumps(rejected, ensure_ascii=False)
    elif choice == 1:
        
        start =random.randint(0,len_sentence)
        end =random.randint(0,len_sentence)
        out = json.dumps([{"start": start, "end": end}], ensure_ascii=False)
    elif choice == 2:
        repeat =random.randint(0,5)
        rejected=[]
        for i in range(repeat):
            start =random.randint(0,len_sentence)
            end =random.randint(start,len_sentence)  
            rejected.append(sentence[start:end])  
        out = json.dumps(rejected, ensure_ascii=False) if rejected else "[]"
    elif choice == 3:
        rejected=[]
        kind = random.choice(["broken_left", "broken_right"])
        start =random.randint(0,len_sentence+1)
        end =random.randint(start,len_sentence+1)
        frag = sentence[start:end]  
        if kind =='broken_right':
           out= f'["{frag}'
        else:
            out= f'{frag}"]'
    elif choice==4:
        rejected=[]
        for gold in chosen:
            rn=random.randint(0,3)
            if rn==0:
                corrupted = left_trim(gold)
                rejected.append(corrupted)
            elif rn==1:
                corrupted =right_trim(gold)
                rejected.append(corrupted)
            else:
                corrupted = both_trim(gold)
                rejected.append(corrupted)
        out= json.dumps(rejected, ensure_ascii=False) if rejected else "[]"
    if out.strip() == chosen.strip():
        out =get_negative_sample(sentence,chosen)
    return out

In [ ]:
from datasets import concatenate_datasets,load_dataset
data_low=load_dataset("haeunkim/curriculum_learning",split='train_low')
data_medium=load_dataset("haeunkim/curriculum_learning",split='train_medium')
data_high=load_dataset("haeunkim/curriculum_learning",split='train_high')
data_low_after=concatenate_datasets([data_low,data_medium,data_high])


In [250]:
random.seed(42)
def add_negative_row(example):
    return {
        "rejected": get_negative_sample(
            sentence=example["input"],
            chosen=example["output"]
        )
    }

processed_data = data_low.map(add_negative_row, desc="Generating negative samples")

processed_medium = data_medium.map(add_negative_row, desc="Generating negative samples")

processed_test = data_high.map(add_negative_row, desc="Generating negative samples")

Generating negative samples:   0%|          | 0/99450 [00:00<?, ? examples/s]

Generating negative samples:   0%|          | 0/99195 [00:00<?, ? examples/s]

Generating negative samples:   0%|          | 0/96110 [00:00<?, ? examples/s]

In [ ]:
from datasets import DatasetDict, Value
dset = DatasetDict({
    "train_low":    processed_data,    
    "train_medium": processed_medium,  
    "train_high":   processed_test,    
})
repo_id = "haeunkim/curriculum_learning_dpo_created_negative"
dset.push_to_hub(
    repo_id,
    private=True,              
        

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/100 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/100 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/97 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/haeunkim/curriculum_learning_dpo_created_negative/commit/449176960cf40a59569b88b1b5cf87c38b43a9c1', commit_message='Upload dataset', commit_description='', oid='449176960cf40a59569b88b1b5cf87c38b43a9c1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/haeunkim/curriculum_learning_dpo_created_negative', endpoint='https://huggingface.co', repo_type='dataset', repo_id='haeunkim/curriculum_learning_dpo_created_negative'), pr_revision=None, pr_num=None)

In [211]:
ap=['I like apple',' "you like", a pear']
for span in ap:
    print(span.split())

['I', 'like', 'apple']
['"you', 'like",', 'a', 'pear']
